In [1]:
%cd /home/ubuntu/eda/flood-risk/data

/home/ubuntu/eda/flood-risk/data


In [1]:
from osgeo import gdal, gdal_array, osr, ogr
import numpy as np
import rasterio
from rasterio import mask
import pyproj
from affine import Affine
import pandas as pd
from shapely.geometry import Point
from geopandas import GeoDataFrame
import geopandas as gpd
from rasterio.features import shapes
import xarray
import matplotlib.pyplot as plt
import rtree
import shapely
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
%cd /home/ubuntu/eda/data

/home/ubuntu/eda/data


In [ ]:
# dataset = rasterio.open('./raster_data/NYC_2017_LiDAR_LandCover.img')

## Read in census block and census tract data

In [4]:
## read in census block shapefile as geodataframe
cb_df = gpd.read_file('./shapefiles/nycb2020_22b/nycb2020.shp')
cb_df.columns = cb_df.columns.str.lower()

ct_df = gpd.read_file('./shapefiles/nyct2020_22b/nyct2020.shp')
ct_df.columns = ct_df.columns.str.lower()

In [5]:
cb_df.groupby('boroname')['bctcb2020'].nunique()

boroname
Bronx             5295
Brooklyn          9769
Manhattan         3930
Queens           13973
Staten Island     4622
Name: bctcb2020, dtype: int64

In [8]:
cb_df.head()

,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,"POLYGON ((973172.666 194632.348, 973310.630 19..."
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,"POLYGON ((972081.788 190733.467, 972184.766 19..."
2,1000,1,Manhattan,000201,10002011000,360610002011000,1569.384823,1.292763e+05,"POLYGON ((988376.731 199328.618, 987837.811 19..."
3,1001,1,Manhattan,000201,10002011001,360610002011001,1594.262855,1.393604e+05,"POLYGON ((988392.400 199070.298, 988285.301 19..."
4,2000,1,Manhattan,000201,10002012000,360610002012000,2055.295961,2.633084e+05,"POLYGON ((988422.186 198807.188, 988449.289 19..."


In [9]:
ct_df.head()

,ctlabel,borocode,boroname,ct2020,boroct2020,cdeligibil,ntaname,nta2020,cdta2020,cdtaname,geoid,shape_leng,shape_area,geometry
0,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,11023.048501,1.844421e+06,"MULTIPOLYGON (((972081.788 190733.467, 972184...."
1,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4754.495244,9.723122e+05,"POLYGON ((988548.218 197770.375, 987978.808 19..."
2,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,6976.286456,2.582705e+06,"POLYGON ((986961.185 199553.643, 987206.139 19..."
3,13,1,Manhattan,001300,1001300,None,Financial District-Battery Park City,MN0101,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061001300,8814.263172,3.412791e+06,"POLYGON ((981865.556 198455.991, 981737.512 19..."
4,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06,"POLYGON ((987475.016 200297.218, 987705.443 20..."


In [10]:
cb_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 37589 entries, 0 to 37588
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   cb2020      37589 non-null  object  
 1   borocode    37589 non-null  object  
 2   boroname    37589 non-null  object  
 3   ct2020      37589 non-null  object  
 4   bctcb2020   37589 non-null  object  
 5   geoid       37589 non-null  object  
 6   shape_leng  37589 non-null  float64 
 7   shape_area  37589 non-null  float64 
 8   geometry    37589 non-null  geometry
dtypes: float64(2), geometry(1), object(6)
memory usage: 2.6+ MB


In [11]:
ct_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2325 entries, 0 to 2324
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ctlabel     2325 non-null   object  
 1   borocode    2325 non-null   object  
 2   boroname    2325 non-null   object  
 3   ct2020      2325 non-null   object  
 4   boroct2020  2325 non-null   object  
 5   cdeligibil  0 non-null      object  
 6   ntaname     2325 non-null   object  
 7   nta2020     2325 non-null   object  
 8   cdta2020    2325 non-null   object  
 9   cdtaname    2325 non-null   object  
 10  geoid       2325 non-null   object  
 11  shape_leng  2325 non-null   float64 
 12  shape_area  2325 non-null   float64 
 13  geometry    2325 non-null   geometry
dtypes: float64(2), geometry(1), object(11)
memory usage: 254.4+ KB


### Get test subset: Manhattan

In [13]:
test = cb_df[cb_df.boroname == 'Manhattan'].reset_index(drop = True)

In [14]:
test

,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,"POLYGON ((973172.666 194632.348, 973310.630 19..."
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,"POLYGON ((972081.788 190733.467, 972184.766 19..."
2,1000,1,Manhattan,000201,10002011000,360610002011000,1569.384823,1.292763e+05,"POLYGON ((988376.731 199328.618, 987837.811 19..."
3,1001,1,Manhattan,000201,10002011001,360610002011001,1594.262855,1.393604e+05,"POLYGON ((988392.400 199070.298, 988285.301 19..."
4,2000,1,Manhattan,000201,10002012000,360610002012000,2055.295961,2.633084e+05,"POLYGON ((988422.186 198807.188, 988449.289 19..."
...,...,...,...,...,...,...,...,...,...
3925,4000,1,Manhattan,004100,10041004000,360610041004000,1382.275259,1.090551e+05,"POLYGON ((985164.480 201755.914, 985350.254 20..."
3926,6004,1,Manhattan,004100,10041006004,360610041006004,956.004136,3.658920e+04,"POLYGON ((985427.596 201917.480, 985398.437 20..."
3927,3001,1,Manhattan,022700,10227003001,360610227003001,2724.638115,3.062444e+05,"POLYGON ((999792.446 239359.284, 999720.478 23..."
3928,2000,1,Manhattan,025900,10259002000,360610259002000,1226.289204,9.308197e+04,"POLYGON ((1000325.578 239064.410, 1000188.288 ..."


In [15]:
test_ct = ct_df[ct_df.boroname == 'Manhattan'].reset_index(drop = True)
test_ct

,ctlabel,borocode,boroname,ct2020,boroct2020,cdeligibil,ntaname,nta2020,cdta2020,cdtaname,geoid,shape_leng,shape_area,geometry
0,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,11023.048501,1.844421e+06,"MULTIPOLYGON (((972081.788 190733.467, 972184...."
1,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4754.495244,9.723122e+05,"POLYGON ((988548.218 197770.375, 987978.808 19..."
2,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,6976.286456,2.582705e+06,"POLYGON ((986961.185 199553.643, 987206.139 19..."
3,13,1,Manhattan,001300,1001300,None,Financial District-Battery Park City,MN0101,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061001300,8814.263172,3.412791e+06,"POLYGON ((981865.556 198455.991, 981737.512 19..."
4,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06,"POLYGON ((987475.016 200297.218, 987705.443 20..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,10.02,1,Manhattan,001002,1001002,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001002,6239.544831,2.195050e+06,"POLYGON ((991575.973 201081.976, 991529.136 20..."
306,239,1,Manhattan,023900,1023900,None,Washington Heights (South),MN1201,MN12,MN12 Washington Heights-Inwood (CD 12 Equivalent),36061023900,3637.448415,6.877886e+05,"POLYGON ((1000753.812 243053.883, 1001087.456 ..."
307,311,1,Manhattan,031100,1031100,None,Highbridge Park,MN1291,MN12,MN12 Washington Heights-Inwood (CD 12 Equivalent),36061031100,27245.589091,7.752363e+06,"POLYGON ((1005732.070 251035.202, 1005745.488 ..."
308,227,1,Manhattan,022700,1022700,None,Hamilton Heights-Sugar Hill,MN0903,MN09,MN09 Morningside Heights-Hamilton Heights (CD ...,36061022700,5486.855748,1.621656e+06,"POLYGON ((1000046.910 239218.373, 999960.597 2..."


In [16]:
# print(test.loc[0])
# rasterio.features.geometry_window(dataset,[test['geometry']])

In [17]:
# out_im, out_trns = mask.mask(dataset, [test['geometry'][0]], crop = True)

In [18]:
# np.histogram(out_im, bins=[0, 1, 2, 3,4,5,6,7,8,9])[0]

array([5886215, 1234434, 1056660,    9786,   20073, 1275527,    7610,
       1211075,       0])

In [19]:
# sum(np.histogram(out_im, bins=[0, 1, 2, 3,4,5,6,7,8])[0])

10701380

In [20]:
# out_im.shape[1]*out_im.shape[2]

10701380

In [21]:
# out_im.shape

(1, 3235, 3308)

In [22]:
# out_trns

Affine(0.5, 0.0, 972139.43,
       0.0, -0.5, 194632.8)

In [23]:
# #get offset in pixels 
# x_offset, y_offset = get_coords_from_lat_long(lat, long, geotransform)

# # get polygons for visualizing
# results = ({'properties': {'raster_val': v}, 'geometry': s}
#            for i, (s, v) in enumerate(
#             shapes(lc_array, mask=None,
#                    transform=Affine(0.5, 0.0, 912286.93+x_offset/2,0.0, -0.5, 273618.3-y_offset/2))))
# count = 0
# for item in results:
#     print(item)
#     count +=1
#     if count ==2:
#         break
# # #convert to geopandas df
# # geoms = list(results)
# # gdf_p  = GeoDataFrame.from_features(geoms, crs = 'EPSG:2263')

Land Cover Mappings
- 0: ? Emailed NYC DOITT - waiting for response
- 1: Tree Canopy
- 2: Grass/Shrub
- 3: Bare Soil
- 4: Water
- 5: Buildings
- 6: Roads
- 7: Other Impervious
- 8: Railroads

## Land Cover Features

In [4]:
dataset = rasterio.open('./raster_data/NYC_2017_LiDAR_LandCover.img')

In [24]:
%%time
test['land_cvr_0'] = 0
test['land_cvr_1'] = 0
test['land_cvr_2'] = 0
test['land_cvr_3'] = 0
test['land_cvr_4'] = 0
test['land_cvr_5'] = 0
test['land_cvr_6'] = 0
test['land_cvr_7'] = 0
test['land_cvr_8'] = 0

# test['transform'] = 0
for i in range(0, len(test)):
    out_im, out_trns = mask.mask(dataset, [test['geometry'][i]], crop = True)
    lc_proportion = np.histogram(out_im, bins=[0, 1, 2, 3,4,5,6,7,8,9])[0]/(out_im.shape[1]*out_im.shape[2])
    for j in range(0,len(lc_proportion)):
        var_name = 'land_cvr_'+str(j)
        # print(var_name)
        # print(lc_proportion[j])
        test.loc[i, var_name] = round(lc_proportion[j],6)
    # test.loc[i, 'transform'] = [out_trns]
    
test.head()

CPU times: user 2min 46s, sys: 5.13 s, total: 2min 51s
Wall time: 2min 51s


,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry,land_cvr_0,land_cvr_1,land_cvr_2,land_cvr_3,land_cvr_4,land_cvr_5,land_cvr_6,land_cvr_7,land_cvr_8
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,"POLYGON ((973172.666 194632.348, 973310.630 19...",0.550043,0.115353,0.098741,0.000914,0.001876,0.119193,0.000711,0.113170,0.0
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,"POLYGON ((972081.788 190733.467, 972184.766 19...",0.531279,0.098717,0.107690,0.033843,0.004596,0.071729,0.000000,0.152147,0.0
2,1000,1,Manhattan,000201,10002011000,360610002011000,1569.384823,1.292763e+05,"POLYGON ((988376.731 199328.618, 987837.811 19...",0.251765,0.079286,0.041528,0.000000,0.000000,0.280771,0.144003,0.202646,0.0
3,1001,1,Manhattan,000201,10002011001,360610002011001,1594.262855,1.393604e+05,"POLYGON ((988392.400 199070.298, 988285.301 19...",0.221387,0.142834,0.008955,0.000000,0.000000,0.301834,0.148332,0.176659,0.0
4,2000,1,Manhattan,000201,10002012000,360610002012000,2055.295961,2.633084e+05,"POLYGON ((988422.186 198807.188, 988449.289 19...",0.185888,0.483037,0.029179,0.001401,0.000000,0.080510,0.118579,0.101406,0.0


In [25]:
test.mean()

/tmp/ipykernel_4074/1706541227.py:1: FutureWarning: The default value of numeric_only in GeoDataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test.mean()


cb2020                 inf
borocode               inf
ct2020                 inf
bctcb2020              inf
geoid                  inf
shape_leng    1.779044e+03
shape_area    1.619862e+05
land_cvr_0    5.514671e-01
land_cvr_1    6.887920e-02
land_cvr_2    1.121019e-02
land_cvr_3    7.826461e-04
land_cvr_4    9.759468e-04
land_cvr_5    1.642570e-01
land_cvr_6    1.004407e-01
land_cvr_7    1.005995e-01
land_cvr_8    1.387682e-03
dtype: float64

In [26]:
test[:10].explore('land_cvr_0', legend = False)

In [27]:
%%time
test['land_cvr_1_rb'] = 0
test['land_cvr_2_rb'] = 0
test['land_cvr_3_rb'] = 0
test['land_cvr_4_rb'] = 0
test['land_cvr_5_rb'] = 0
test['land_cvr_6_rb'] = 0
test['land_cvr_7_rb'] = 0
test['land_cvr_8_rb'] = 0

# test['transform'] = 0
for i in range(0, len(test)):
    # out_im, out_trns = mask.mask(dataset, [test['geometry'][i]], crop = True)
    # lc_proportion = np.histogram(out_im, bins=[0, 1, 2, 3,4,5,6,7,8,9])[0]/(out_im.shape[1]*out_im.shape[2])
    for j in range(1,9):
        new_var_name = 'land_cvr_'+str(j)+'_rb'
        var_name = 'land_cvr_'+str(j)
        # print(var_name)
        # print(lc_proportion[j])
        test.loc[i, new_var_name] = round(test.loc[i,var_name]/(1-test.loc[i,'land_cvr_0']),6)
    # test.loc[i, 'transform'] = [out_trns]
    
test.head()

CPU times: user 9.93 s, sys: 0 ns, total: 9.93 s
Wall time: 9.93 s


,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry,land_cvr_0,...,land_cvr_7,land_cvr_8,land_cvr_1_rb,land_cvr_2_rb,land_cvr_3_rb,land_cvr_4_rb,land_cvr_5_rb,land_cvr_6_rb,land_cvr_7_rb,land_cvr_8_rb
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,"POLYGON ((973172.666 194632.348, 973310.630 19...",0.550043,...,0.113170,0.0,0.256364,0.219445,0.002031,0.004169,0.264899,0.001580,0.251513,0.0
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,"POLYGON ((972081.788 190733.467, 972184.766 19...",0.531279,...,0.152147,0.0,0.210609,0.229753,0.072203,0.009805,0.153031,0.000000,0.324600,0.0
2,1000,1,Manhattan,000201,10002011000,360610002011000,1569.384823,1.292763e+05,"POLYGON ((988376.731 199328.618, 987837.811 19...",0.251765,...,0.202646,0.0,0.105964,0.055501,0.000000,0.000000,0.375244,0.192457,0.270832,0.0
3,1001,1,Manhattan,000201,10002011001,360610002011001,1594.262855,1.393604e+05,"POLYGON ((988392.400 199070.298, 988285.301 19...",0.221387,...,0.176659,0.0,0.183447,0.011501,0.000000,0.000000,0.387656,0.190508,0.226889,0.0
4,2000,1,Manhattan,000201,10002012000,360610002012000,2055.295961,2.633084e+05,"POLYGON ((988422.186 198807.188, 988449.289 19...",0.185888,...,0.101406,0.0,0.593330,0.035842,0.001721,0.000000,0.098893,0.145654,0.124560,0.0


In [28]:
#remove lc raster from env to save space
del dataset

## Elevation Features

#### Census Block Level

In [29]:
## read in elevation raster
elev_path = './elevation/DEM_LiDAR_1ft_2010_Improved_NYC_int.tif'
elev_data = rasterio.open(elev_path)

In [30]:
out_im, out_trns = mask.mask(elev_data, [test['geometry'][0]], crop = True)

In [31]:
out_im

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]], dtype=uint16)

In [32]:
out_im.shape

(1, 1617, 1654)

In [33]:
out_im.mean()

3.078037238859488

In [34]:
out_im.min()

0

In [35]:
out_im.max()

14

In [36]:
np.histogram(out_im)

(array([1484138,    1977,    6322,    7640,  425033,  668722,   57222,
          17406,    2857,    3201]),
 array([ 0. ,  1.4,  2.8,  4.2,  5.6,  7. ,  8.4,  9.8, 11.2, 12.6, 14. ]))

In [37]:
np.percentile(out_im, [0,25,75,100])

array([ 0.,  0.,  7., 14.])

In [38]:
sum(np.histogram(out_im)[0])

2674518

In [39]:
out_im.shape[1]*out_im.shape[2]

2674518

In [40]:
%%time
test['avg_elev'] = 0
test['min_elev'] = 0
test['max_elev'] = 0
test['q1_elev'] = 0
test['q3_elev'] = 0


# test['transform'] = 0
for i in range(0, len(test)):
    out_im, out_trns = mask.mask(elev_data, [test['geometry'][i]], crop = True)
    test.loc[i, 'avg_elev'] = round(out_im.mean(),1)
    test.loc[i, 'min_elev'], test.loc[i, 'q1_elev'], test.loc[i, 'q3_elev'], test.loc[i, 'max_elev'] = np.percentile(out_im, [0,25,75,100])
    
test.head()

CPU times: user 2min 44s, sys: 3.4 s, total: 2min 47s
Wall time: 2min 47s


,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry,land_cvr_0,...,land_cvr_4_rb,land_cvr_5_rb,land_cvr_6_rb,land_cvr_7_rb,land_cvr_8_rb,avg_elev,min_elev,max_elev,q1_elev,q3_elev
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,"POLYGON ((973172.666 194632.348, 973310.630 19...",0.550043,...,0.004169,0.264899,0.001580,0.251513,0.0,3.1,0,14,0,7
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,"POLYGON ((972081.788 190733.467, 972184.766 19...",0.531279,...,0.009805,0.153031,0.000000,0.324600,0.0,5.1,0,20,0,11
2,1000,1,Manhattan,000201,10002011000,360610002011000,1569.384823,1.292763e+05,"POLYGON ((988376.731 199328.618, 987837.811 19...",0.251765,...,0.000000,0.375244,0.192457,0.270832,0.0,22.8,0,37,0,31
3,1001,1,Manhattan,000201,10002011001,360610002011001,1594.262855,1.393604e+05,"POLYGON ((988392.400 199070.298, 988285.301 19...",0.221387,...,0.000000,0.387656,0.190508,0.226889,0.0,22.6,0,35,26,31
4,2000,1,Manhattan,000201,10002012000,360610002012000,2055.295961,2.633084e+05,"POLYGON ((988422.186 198807.188, 988449.289 19...",0.185888,...,0.000000,0.098893,0.145654,0.124560,0.0,20.3,0,36,20,27


#### Census Tract Level
Can use to look at if block is local minimum/maximum in area

In [41]:
%%time
test_ct['avg_elev_ct'] = 0
test_ct['min_elev_ct'] = 0
test_ct['max_elev_ct'] = 0
test_ct['q1_elev_ct'] = 0
test_ct['q3_elev_ct'] = 0


# test['transform'] = 0
for i in range(0, len(test_ct)):
    out_im, out_trns = mask.mask(elev_data, [test_ct['geometry'][i]], crop = True)
    test_ct.loc[i, 'avg_elev_ct'] = round(out_im.mean(),1)
    test_ct.loc[i, 'min_elev_ct'], test_ct.loc[i, 'q1_elev_ct'], test_ct.loc[i, 'q3_elev_ct'], test_ct.loc[i, 'max_elev_ct'] = np.percentile(out_im, [0,25,75,100])
    
test_ct.head()

CPU times: user 2min 27s, sys: 2.07 s, total: 2min 29s
Wall time: 2min 29s


,ctlabel,borocode,boroname,ct2020,boroct2020,cdeligibil,ntaname,nta2020,cdta2020,cdtaname,geoid,shape_leng,shape_area,geometry,avg_elev_ct,min_elev_ct,max_elev_ct,q1_elev_ct,q3_elev_ct
0,1,1,Manhattan,000100,1000100,None,The Battery-Governors Island-Ellis Island-Libe...,MN0191,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061000100,11023.048501,1.844421e+06,"MULTIPOLYGON (((972081.788 190733.467, 972184....",1.2,0,20,0,0
1,2.01,1,Manhattan,000201,1000201,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000201,4754.495244,9.723122e+05,"POLYGON ((988548.218 197770.375, 987978.808 19...",14.0,0,37,0,27
2,6,1,Manhattan,000600,1000600,None,Chinatown-Two Bridges,MN0301,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061000600,6976.286456,2.582705e+06,"POLYGON ((986961.185 199553.643, 987206.139 19...",13.5,0,42,6,23
3,13,1,Manhattan,001300,1001300,None,Financial District-Battery Park City,MN0101,MN01,MN01 Financial District-Tribeca (CD 1 Equivalent),36061001300,8814.263172,3.412791e+06,"POLYGON ((981865.556 198455.991, 981737.512 19...",6.2,0,41,0,9
4,14.01,1,Manhattan,001401,1001401,None,Lower East Side,MN0302,MN03,MN03 Lower East Side-Chinatown (CD 3 Equivalent),36061001401,5075.332000,1.006117e+06,"POLYGON ((987475.016 200297.218, 987705.443 20...",17.3,0,44,0,34


In [42]:
test = test.merge(test_ct[['ct2020','ntaname','avg_elev_ct','min_elev_ct','max_elev_ct','q1_elev_ct','q3_elev_ct']],
           on = 'ct2020')

In [43]:
len(test.cb2020.unique())

138

In [44]:
len(test.ct2020.unique())

310

In [45]:
len(test.bctcb2020.unique())

3930

## Catch Basin Features

How to consider interactions? (ex. is the census block a local minimum in its census tract and does it have low catch basin density)

In [58]:
dr_path = './DEPCatchbasins/DEPCATCHBASINS.shp'
dr_df = gpd.read_file(dr_path)
dr_df.columns = dr_df.columns.str.lower()
dr_df.head()

,unitid,latitude,longitude,point_x,point_y,geometry
0,CB550115,40.518700,-74.194200,930230.623100,128315.947500,POINT (930230.623 128315.948)
1,CB550005,40.518580,-74.194200,930232.445300,128271.111900,POINT (930232.445 128271.112)
2,CB550061,40.518062,-74.195963,929765.086058,128081.621106,POINT (929765.086 128081.621)
3,CB550062,40.517990,-74.195894,929784.007040,128055.367355,POINT (929784.007 128055.367)
4,CB550064,40.518575,-74.196172,929707.412976,128268.675615,POINT (929707.413 128268.676)


In [59]:
len(dr_df)

153372

In [60]:
dr_df.unitid.nunique()

153372

In [61]:
dr_df = dr_df.sjoin(test, how = 'inner', predicate = 'within')[['unitid','bctcb2020']]

In [62]:
test = test.merge(dr_df.groupby('bctcb2020')['unitid'].nunique(),how = 'left', on = 'bctcb2020').rename(columns = {'unitid':'catch_basin_count'})

In [63]:
test.head()

,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry,land_cvr_0,...,max_elev,q1_elev,q3_elev,ntaname,avg_elev_ct,min_elev_ct,max_elev_ct,q1_elev_ct,q3_elev_ct,catch_basin_count
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,"POLYGON ((973172.666 194632.348, 973310.630 19...",0.550043,...,14,0,7,The Battery-Governors Island-Ellis Island-Libe...,1.2,0,20,0,0,NaN
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,"POLYGON ((972081.788 190733.467, 972184.766 19...",0.531279,...,20,0,11,The Battery-Governors Island-Ellis Island-Libe...,1.2,0,20,0,0,NaN
2,1000,1,Manhattan,000201,10002011000,360610002011000,1569.384823,1.292763e+05,"POLYGON ((988376.731 199328.618, 987837.811 19...",0.251765,...,37,0,31,Chinatown-Two Bridges,14.0,0,37,0,27,2.0
3,1001,1,Manhattan,000201,10002011001,360610002011001,1594.262855,1.393604e+05,"POLYGON ((988392.400 199070.298, 988285.301 19...",0.221387,...,35,26,31,Chinatown-Two Bridges,14.0,0,37,0,27,2.0
4,2000,1,Manhattan,000201,10002012000,360610002012000,2055.295961,2.633084e+05,"POLYGON ((988422.186 198807.188, 988449.289 19...",0.185888,...,36,20,27,Chinatown-Two Bridges,14.0,0,37,0,27,3.0


In [64]:
test['catch_basin_density'] = test['catch_basin_count']/test['shape_area']

In [65]:
test.head()

,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry,land_cvr_0,...,q1_elev,q3_elev,ntaname,avg_elev_ct,min_elev_ct,max_elev_ct,q1_elev_ct,q3_elev_ct,catch_basin_count,catch_basin_density
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,"POLYGON ((973172.666 194632.348, 973310.630 19...",0.550043,...,0,7,The Battery-Governors Island-Ellis Island-Libe...,1.2,0,20,0,0,NaN,NaN
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,"POLYGON ((972081.788 190733.467, 972184.766 19...",0.531279,...,0,11,The Battery-Governors Island-Ellis Island-Libe...,1.2,0,20,0,0,NaN,NaN
2,1000,1,Manhattan,000201,10002011000,360610002011000,1569.384823,1.292763e+05,"POLYGON ((988376.731 199328.618, 987837.811 19...",0.251765,...,0,31,Chinatown-Two Bridges,14.0,0,37,0,27,2.0,0.000015
3,1001,1,Manhattan,000201,10002011001,360610002011001,1594.262855,1.393604e+05,"POLYGON ((988392.400 199070.298, 988285.301 19...",0.221387,...,26,31,Chinatown-Two Bridges,14.0,0,37,0,27,2.0,0.000014
4,2000,1,Manhattan,000201,10002012000,360610002012000,2055.295961,2.633084e+05,"POLYGON ((988422.186 198807.188, 988449.289 19...",0.185888,...,20,27,Chinatown-Two Bridges,14.0,0,37,0,27,3.0,0.000011


## Subway Features

In [66]:
se_path = './doitt_subway_entrances/DOITT_SUBWAY_ENTRANCE_04JAN2017.shp'
se_df = gpd.read_file(se_path)
se_df.columns = se_df.columns.str.lower()
se_df.head()

,objectid,name,url,line,geometry
0,1734,Birchall Ave & Sagamore St at NW corner,http://web.mta.info/nyct/service/,2-5,POINT (1020670.669 248679.991)
1,1735,Birchall Ave & Sagamore St at NE corner,http://web.mta.info/nyct/service/,2-5,POINT (1020710.254 248665.112)
2,1736,Morris Park Ave & 180th St at NW corner,http://web.mta.info/nyct/service/,2-5,POINT (1019251.986 245782.870)
3,1737,Morris Park Ave & 180th St at NW corner,http://web.mta.info/nyct/service/,2-5,POINT (1019419.818 245866.910)
4,1738,Boston Rd & 178th St at SW corner,http://web.mta.info/nyct/service/,2-5,POINT (1017557.722 245631.832)


In [67]:
se_df = se_df.sjoin(test, how = 'inner', predicate = 'within')[['objectid','bctcb2020']]

In [68]:
test = test.merge(se_df.groupby('bctcb2020')['objectid'].nunique(),how = 'left', on = 'bctcb2020').rename(columns = {'objectid':'subway_entrance_count'})

In [69]:
test.head()

,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry,land_cvr_0,...,q3_elev,ntaname,avg_elev_ct,min_elev_ct,max_elev_ct,q1_elev_ct,q3_elev_ct,catch_basin_count,catch_basin_density,subway_entrance_count
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,"POLYGON ((973172.666 194632.348, 973310.630 19...",0.550043,...,7,The Battery-Governors Island-Ellis Island-Libe...,1.2,0,20,0,0,NaN,NaN,NaN
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,"POLYGON ((972081.788 190733.467, 972184.766 19...",0.531279,...,11,The Battery-Governors Island-Ellis Island-Libe...,1.2,0,20,0,0,NaN,NaN,NaN
2,1000,1,Manhattan,000201,10002011000,360610002011000,1569.384823,1.292763e+05,"POLYGON ((988376.731 199328.618, 987837.811 19...",0.251765,...,31,Chinatown-Two Bridges,14.0,0,37,0,27,2.0,0.000015,NaN
3,1001,1,Manhattan,000201,10002011001,360610002011001,1594.262855,1.393604e+05,"POLYGON ((988392.400 199070.298, 988285.301 19...",0.221387,...,31,Chinatown-Two Bridges,14.0,0,37,0,27,2.0,0.000014,NaN
4,2000,1,Manhattan,000201,10002012000,360610002012000,2055.295961,2.633084e+05,"POLYGON ((988422.186 198807.188, 988449.289 19...",0.185888,...,27,Chinatown-Two Bridges,14.0,0,37,0,27,3.0,0.000011,NaN


In [71]:
test['subway_entrance_count'].describe()

count    487.000000
mean       1.591376
std        0.865055
min        1.000000
25%        1.000000
50%        1.000000
75%        2.000000
max        6.000000
Name: subway_entrance_count, dtype: float64

## Retaining Wall Features

In [14]:
rw_path = './retaining_wall/RETAININGWALL.shp'
rw_df = gpd.read_file(rw_path)
rw_df.columns = rw_df.columns.str.lower()

In [15]:
rw_df.head()

,source_id,feat_code,sub_code,status,shape_leng,geometry
0,2.140000e+10,4000,400000,Unchanged,365.715400,"LINESTRING Z (985530.573 194711.453 44.408, 98..."
1,2.140000e+10,4000,400000,Unchanged,226.155070,"LINESTRING Z (985475.003 194408.890 68.260, 98..."
2,2.140000e+10,4000,400000,Unchanged,50.949832,"LINESTRING Z (985523.914 194769.822 22.721, 98..."
3,2.140000e+10,4000,400000,Unchanged,175.276149,"LINESTRING Z (987357.515 194593.661 68.980, 98..."
4,2.140000e+10,4000,400000,Unchanged,100.051403,"LINESTRING Z (987949.170 194143.960 70.970, 98..."


In [87]:
for i in range(0,len(test)):
    # rw_check = test.loc[[i]].sjoin(rw_df, how = 'inner', predicate = 'intersects')
    rw_check = test.loc[[i]].overlay(rw_df, how = 'intersection', keep_geom_type = False)

    if len(rw_check) > 0:
        print(i)
        print(rw_check)
        break

54
  cb2020 borocode   boroname  ct2020    bctcb2020            geoid  \
0   1024        1  Manhattan  000500  10005001024  360610005001024   
1   1024        1  Manhattan  000500  10005001024  360610005001024   
2   1024        1  Manhattan  000500  10005001024  360610005001024   
3   1024        1  Manhattan  000500  10005001024  360610005001024   
4   1024        1  Manhattan  000500  10005001024  360610005001024   

   shape_leng_1    shape_area  land_cvr_0  land_cvr_1  ...  q3_elev_ct  \
0   5495.429939  1.756075e+06    0.384133    0.155764  ...           7   
1   5495.429939  1.756075e+06    0.384133    0.155764  ...           7   
2   5495.429939  1.756075e+06    0.384133    0.155764  ...           7   
3   5495.429939  1.756075e+06    0.384133    0.155764  ...           7   
4   5495.429939  1.756075e+06    0.384133    0.155764  ...           7   

   catch_basin_count  catch_basin_density  subway_entrance_count  \
0                NaN                  NaN                    Na

In [93]:
%%time
rw_df = test.overlay(rw_df, how = 'intersection', keep_geom_type = False)[['bctcb2020','feat_code','status','shape_leng_2','geometry']]
rw_df

CPU times: user 2.08 s, sys: 50 µs, total: 2.08 s
Wall time: 2.08 s


,bctcb2020,feat_code,status,shape_leng_2,geometry
0,10005001024,4000,New,313.388625,"LINESTRING Z (980033.596 191386.965 30.364, 98..."
1,10005001024,4000,New,273.102243,"LINESTRING Z (980033.850 191115.752 31.834, 98..."
2,10005001024,4000,New,1108.066856,"LINESTRING Z (979667.792 191237.879 45.346, 97..."
3,10005001024,4000,New,1070.475907,"LINESTRING Z (979957.510 191118.600 45.147, 97..."
4,10005001024,4000,New,260.744595,"LINESTRING Z (979759.247 191422.410 37.502, 97..."
...,...,...,...,...,...
398,10311001009,4000,New,47.232091,"LINESTRING Z (1003279.368 247417.797 121.465, ..."
399,10311001009,4000,New,52.788095,"LINESTRING Z (1003474.347 247417.286 134.378, ..."
400,10317033011,4000,New,180.254565,"LINESTRING Z (979598.919 199860.768 31.764, 97..."
401,10317033011,4000,New,196.639198,"LINESTRING Z (979598.919 199860.768 31.764, 97..."


In [94]:
rw_df.groupby('bctcb2020')['shape_leng_2'].sum()

bctcb2020
10005001024    3025.778227
10013002004     331.525499
10015012002     114.290959
10016001003     211.466144
10016001004     259.932311
                  ...     
10311001000    2132.362095
10311001003    1073.543383
10311001009    3629.881778
10317033011     376.893764
10317043005     327.249538
Name: shape_leng_2, Length: 173, dtype: float64

In [133]:
length = []
avg_rw_elev = []
cb_list = []
for cb in rw_df.bctcb2020.unique():
    cb_list.append(cb)
    elev_list = []
    subset = rw_df[rw_df.bctcb2020 == cb]
    length.append(subset['shape_leng_2'].sum())
    for i in range(0,len(subset)):
        for j in list(subset.iloc[i].geometry.coords):
             elev_list.append(j[2])
    avg_rw_elev.append(np.mean(elev_list))

In [137]:
rw_agg = pd.DataFrame(zip(cb_list, length, avg_rw_elev),columns  = ['bctcb2020','rw_length','rw_avg_elev'])
rw_agg.head()

,bctcb2020,rw_length,rw_avg_elev
0,10005001024,3025.778227,35.128024
1,10013002004,331.525499,7.606025
2,10015012002,114.290959,34.750000
3,10016001003,211.466144,57.276600
4,10016001004,259.932311,54.952117


In [139]:
test = test.merge(rw_agg, how = 'left', on = 'bctcb2020')

In [140]:
test.rw_length.describe()

count     173.000000
mean      728.938031
std       913.059801
min        17.668380
25%       179.880667
50%       376.893764
75%       784.590156
max      4794.696482
Name: rw_length, dtype: float64

##### Update: RW with Multipart Geometries

In [19]:
test2 = cb_df[cb_df.boroname == 'Brooklyn'].reset_index(drop = True)
rw_ = test2.overlay(rw_df, how = 'intersection', keep_geom_type = False)[['bctcb2020','feat_code','status','shape_leng_2','geometry']]
print(rw_.head())
print('Original length:',len(rw_))
print('Length after splitting multipart geometries:',len(rw_.explode()))

     bctcb2020  feat_code     status  shape_leng_2  \
0  30001001002       4000  Unchanged     92.089954   
1  30001001003       4000  Unchanged    106.066077   
2  30001001006       4000  Unchanged    365.715400   
3  30001001006       4000  Unchanged     50.949832   
4  30001001006       4000  Unchanged    137.872450   

                                            geometry  
0  LINESTRING Z (985676.048 194655.471 64.141, 98...  
1  LINESTRING Z (985754.499 194632.984 62.602, 98...  
2  LINESTRING Z (985530.573 194711.453 44.408, 98...  
3  LINESTRING Z (985523.914 194769.822 22.721, 98...  
4  LINESTRING Z (985517.557 194534.250 67.575, 98...  
Original length: 521
Length after splitting multipart geometries: 525


In [17]:
rw_.explode(ignore_index = True)

,bctcb2020,feat_code,status,shape_leng_2,geometry
0,30001001002,4000,Unchanged,92.089954,"LINESTRING Z (985676.048 194655.471 64.141, 98..."
1,30001001003,4000,Unchanged,106.066077,"LINESTRING Z (985754.499 194632.984 62.602, 98..."
2,30001001006,4000,Unchanged,365.715400,"LINESTRING Z (985530.573 194711.453 44.408, 98..."
3,30001001006,4000,Unchanged,50.949832,"LINESTRING Z (985523.914 194769.822 22.721, 98..."
4,30001001006,4000,Unchanged,137.872450,"LINESTRING Z (985517.557 194534.250 67.575, 98..."
...,...,...,...,...,...
520,30426002005,2460,Unchanged,482.959504,"LINESTRING Z (989702.853 159120.863 21.262, 98..."
521,30118001000,2460,Unchanged,360.482770,"LINESTRING Z (981360.810 170450.673 50.345, 98..."
522,30250001000,2460,Unchanged,137.806080,"LINESTRING Z (985685.427 167225.484 51.197, 98..."
523,30250001000,2460,Unchanged,99.324485,"LINESTRING Z (985558.951 167278.821 41.709, 98..."


In [20]:
test = cb_df[cb_df.boroname == 'Manhattan'].reset_index(drop = True)
rw_2 = test.overlay(rw_df, how = 'intersection', keep_geom_type = False)[['bctcb2020','feat_code','status','shape_leng_2','geometry']]
print(rw_2.head())
print('Original length:',len(rw_2))
print('Length after splitting multipart geometries:',len(rw_2.explode()))

     bctcb2020  feat_code status  shape_leng_2  \
0  10005001024       4000    New    313.388625   
1  10005001024       4000    New    273.102243   
2  10005001024       4000    New   1108.066856   
3  10005001024       4000    New   1070.475907   
4  10005001024       4000    New    260.744595   

                                            geometry  
0  LINESTRING Z (980033.596 191386.965 30.364, 98...  
1  LINESTRING Z (980033.850 191115.752 31.834, 98...  
2  LINESTRING Z (979667.792 191237.879 45.346, 97...  
3  LINESTRING Z (979957.510 191118.600 45.147, 97...  
4  LINESTRING Z (979759.247 191422.410 37.502, 97...  
Original length: 403
Length after splitting multipart geometries: 403


## Hydrography Features
[Metadata](https://github.com/CityOfNewYork/nyc-planimetrics/blob/master/Capture_Rules.md#hydrography)

Feature Code Mappings
- 2600: Lake/Reservoir
- 2610: Pond
- 2620: River
- 2630: Stream
- 2640: Wetland/Marsh
- 2650: Beach/Shoreline
- 2660: Bay/Ocean

In [142]:
h_path = './hydro/HYDROGRAPHY.shp'
h_df = gpd.read_file(h_path)
h_df.columns = h_df.columns.str.lower()
h_df

,name,source_id,feat_code,sub_code,status,shape_leng,shape_area,geometry
0,None,2.126100e+10,2610,261000,Unchanged,284.939991,1.893027e+03,"POLYGON ((985358.307 195199.994, 985356.311 19..."
1,None,2.126100e+10,2610,261000,Unchanged,255.632501,2.361146e+03,"POLYGON ((985162.646 194741.685, 985155.380 19..."
2,EAST RIVER,2.126200e+10,2620,262000,Unchanged,482.569042,8.634500e+03,"POLYGON ((985112.730 194774.258, 985113.447 19..."
3,EAST RIVER,2.126000e+10,2620,262000,Updated,101020.920576,7.736593e+07,"POLYGON ((984408.882 192872.101, 984407.899 19..."
4,None,2.126100e+10,2610,261000,New,187.231412,1.668621e+03,"POLYGON ((985260.594 194975.441, 985257.063 19..."
...,...,...,...,...,...,...,...,...
2211,None,2.026400e+10,2640,264000,Unchanged,5257.043923,3.148807e+05,"POLYGON ((1015660.636 190917.825, 1015809.053 ..."
2212,MARSH,1.726400e+10,2640,264000,Updated,1389.729457,4.988805e+04,"POLYGON ((961818.618 152646.560, 961821.769 15..."
2213,None,1.626400e+10,2640,264000,New,1960.060785,1.143331e+05,"POLYGON ((931520.142 163734.961, 931522.975 16..."
2214,None,1.626400e+10,2640,264000,New,552.340034,5.140816e+03,"POLYGON ((931334.722 163189.904, 931337.802 16..."


In [143]:
h_df.name.unique()

array([None, 'EAST RIVER', 'BARBADOES BASIN', 'EAST ROCKAWAY INLET',
       'VERNAM BASIN', 'SOMMERVILLE BASIN', 'NEGRO BAR CHANNEL',
       'HEAD OF BAY', 'ROCKAWAY INLET', 'BROAD CHANNEL', 'ATLANTIC OCEAN',
       'CONCH BASIN', 'BIG MUCKS CREEK', 'MARSH', 'SHEEPSHEAD BAY',
       'SHELL BANK CREEK', 'MILL CREEK', 'MILL BASIN', 'EAST MILL BASIN',
       'ISLAND CHANNEL', 'PLUM BEACH CHANNEL', 'MARINE PARK CREEK',
       'JACKSON HOLE CREEK', 'STRAIGHT CR', 'MANHATTAN BEACH',
       'GRASS HASSOCK CHANNEL', 'WEST POND', 'GRASSY BAY',
       'BIG FISHKILL CHANNEL', 'EAST POND', 'HORSE CHANNEL',
       'JAMAICA BAY', 'THURSTON BASIN', 'CONSELYEAS POND', 'HOOK CREEK',
       'BERGEN BASIN', 'SHELLBANK BASIN', 'POND', 'HAWTREE BASIN',
       'BAISLEY POND', 'KISSENA LAKE', 'MEADOW LAKE', 'FLUSHING CREEK',
       'WHALE CREEK', 'DUTCH KILLS', 'BOWERY BAY', 'LUYSTER CREEK',
       'NEWTOWN CREEK', 'EAST CHANNEL', 'OAKLAND LAKE', 'ALLEY CREEK',
       'LONG ISLAND SOUND', 'LITTLE NECK BAY', 

In [144]:
h_df.feat_code.unique()

array([2610, 2620, 2630, 2640, 2650, 2660, 2600])

In [145]:
%%time
hyd_df = test.overlay(h_df, how = 'intersection', keep_geom_type = False)#[['bctcb2020','feat_code','status','shape_area_2','geometry']]
hyd_df

CPU times: user 2.89 s, sys: 0 ns, total: 2.89 s
Wall time: 2.89 s


,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng_1,shape_area_1,land_cvr_0,land_cvr_1,...,rw_length,rw_avg_elev,name,source_id,feat_code,sub_code,status,shape_leng_2,shape_area_2,geometry
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,0.550043,0.115353,...,NaN,NaN,UPPER NEW YORK BAY,1.626600e+10,2660,266000,Updated,159165.151759,2.042867e+08,"MULTIPOLYGON (((973686.041 193957.699, 973649...."
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,0.531279,0.098717,...,NaN,NaN,UPPER NEW YORK BAY,1.626600e+10,2660,266000,Updated,159165.151759,2.042867e+08,"MULTIPOLYGON (((972184.766 190551.136, 972398...."
2,1001,1,Manhattan,000500,10005001001,360610005001001,23617.573933,5.048118e+05,0.963337,0.007328,...,NaN,NaN,UPPER NEW YORK BAY,1.626600e+10,2660,266000,Updated,159165.151759,2.042867e+08,"MULTIPOLYGON (((978768.243 191785.000, 978769...."
3,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,0.550043,0.115353,...,NaN,NaN,HUDSON RIVER,1.926200e+10,2620,262000,Unchanged,13891.033529,6.730030e+06,"MULTIPOLYGON (((973330.005 194518.265, 973793...."
4,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,0.531279,0.098717,...,NaN,NaN,None,1.926500e+10,2650,265000,Unchanged,162.458071,1.178093e+03,"POLYGON ((971190.557 190854.053, 971186.682 19..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,1005,1,Manhattan,029700,10297001005,360610297001005,7293.943030,8.305477e+05,0.837326,0.038621,...,88.623770,30.726275,SPUYTEN DUYVIL CREEK,1.326200e+10,2620,262000,Updated,14069.607277,2.907756e+06,"POLYGON ((1004615.626 259014.948, 1004614.497 ..."
239,1001,1,Manhattan,029700,10297001001,360610297001001,5637.216802,1.344656e+06,0.450493,0.086991,...,221.317577,46.296294,SPUYTEN DUYVIL CREEK,1.326200e+10,2620,262000,Updated,14069.607277,2.907756e+06,"MULTIPOLYGON (((1008148.633 257720.553, 100817..."
240,1003,1,Manhattan,029700,10297001003,360610297001003,13624.504699,6.253287e+06,0.523487,0.391759,...,309.046043,115.053617,SPUYTEN DUYVIL CREEK,1.326200e+10,2620,262000,Updated,14069.607277,2.907756e+06,"MULTIPOLYGON (((1006103.601 258219.584, 100610..."
241,1000,1,Manhattan,030900,10309001000,360610309001000,2608.368491,3.395500e+05,0.556943,0.012886,...,NaN,NaN,SPUYTEN DUYVIL CREEK,1.326200e+10,2620,262000,Updated,14069.607277,2.907756e+06,"MULTIPOLYGON (((1009150.947 257505.123, 100913..."


In [146]:
hyd_df.bctcb2020.nunique()

210

In [147]:
hyd_df.groupby(['bctcb2020','feat_code'])['shape_area_2'].sum()

bctcb2020    feat_code
10001001000  2620         6.730030e+06
             2660         2.042867e+08
10001001001  2650         1.178093e+03
             2660         2.042867e+08
10002012004  2620         7.736593e+07
                              ...     
10319000001  2620         2.440659e+08
10319000003  2620         3.214318e+08
10319001004  2620         2.440659e+08
10319001005  2620         2.440659e+08
10319001006  2620         7.736593e+07
Name: shape_area_2, Length: 224, dtype: float64

In [154]:
hydro_dict = {2600:'lake_res_ind',
              2610:'pond_ind',
              2620:'river_ind',
              2630:'stream_ind',
              2640:'wl_marsh_ind',
              2650:'beach_shore_ind',
              2660:'bay_ocean_ind'}

In [155]:
for key in hydro_dict.keys():
    test[hydro_dict[key]] = np.where(test.bctcb2020.isin(hyd_df[hyd_df.feat_code == key].bctcb2020),1,0)

In [156]:
test.head()

,cb2020,borocode,boroname,ct2020,bctcb2020,geoid,shape_leng,shape_area,geometry,land_cvr_0,...,subway_entrance_count,rw_length,rw_avg_elev,lake_res_ind,pond_ind,river_ind,stream_ind,wl_marsh_ind,beach_shore_ind,bay_ocean_ind
0,1000,1,Manhattan,000100,10001001000,360610001001000,6627.858318,1.204255e+06,"POLYGON ((973172.666 194632.348, 973310.630 19...",0.550043,...,NaN,NaN,NaN,0,0,1,0,0,0,1
1,1001,1,Manhattan,000100,10001001001,360610001001001,4395.190183,6.401664e+05,"POLYGON ((972081.788 190733.467, 972184.766 19...",0.531279,...,NaN,NaN,NaN,0,0,0,0,0,1,1
2,1000,1,Manhattan,000201,10002011000,360610002011000,1569.384823,1.292763e+05,"POLYGON ((988376.731 199328.618, 987837.811 19...",0.251765,...,NaN,NaN,NaN,0,0,0,0,0,0,0
3,1001,1,Manhattan,000201,10002011001,360610002011001,1594.262855,1.393604e+05,"POLYGON ((988392.400 199070.298, 988285.301 19...",0.221387,...,NaN,NaN,NaN,0,0,0,0,0,0,0
4,2000,1,Manhattan,000201,10002012000,360610002012000,2055.295961,2.633084e+05,"POLYGON ((988422.186 198807.188, 988449.289 19...",0.185888,...,NaN,NaN,NaN,0,0,0,0,0,0,0


In [166]:
test.to_csv('sample.csv')

In [168]:
!aws s3 cp sample.csv s3://w210-flood-risk/modeling_data/sample.csv --acl public-read

upload: ./sample.csv to s3://w210-flood-risk/modeling_data/sample.csv
